# Building a Smart Conversation Manager with Groq AI

**What we are building**: A conversational AI system that can remember what you have talked about and extract useful information from your chats.

**Our goals**:
1. Create a system that can maintain conversation history and summarize long chats
2. Build an information extractor that can find personal details like names, emails, and locations from conversations

**Created by**: Somil Agrawal  
**Date**: September 17, 2025

Let us dive in and build something amazing!

## Getting Started: Setting Up Our AI Tools

First, let us get our development environment ready. We will install the packages we need and connect to Groq AI service to power our conversation system.

In [1]:
# Install the tools we need for our conversation manager
!pip install openai python-dotenv jsonschema

# Now we will import all the Python libraries we will use
import os
import json
import time
from datetime import datetime
from typing import List, Dict, Any, Optional
from openai import OpenAI
import jsonschema
from jsonschema import validate

print("All our tools are ready to go.")

All our tools are ready to go.


In [2]:
# Connect to Groq AI service
# Note: In a real project, you would store this API key securely, not in your code!
GROQ_API_KEY = "your_api_key"

# Setting up our connection to Groq AI
client = OpenAI(
    api_key=GROQ_API_KEY,
    base_url="https://api.groq.com/openai/v1"
)

# Test with current available models
current_models = [
    "llama-3.1-8b-instant",
    "llama-3.1-70b-versatile", 
    "mixtral-8x7b-32768",
    "gemma2-9b-it"
]

DEFAULT_MODEL = None

for model in current_models:
    try:
        print(f"Testing model: {model}")
        test_response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": "Hello, test connection"}],
            max_tokens=10
        )
        print(f"Success! Connected with model: {model}")
        print(f"The AI responded: {test_response.choices[0].message.content}")
        DEFAULT_MODEL = model
        break
    except Exception as e:
        print(f"Model {model} failed: {str(e)[:100]}...")
        continue

if DEFAULT_MODEL:
    print(f"\nPerfect! We will use {DEFAULT_MODEL} for our conversations.")
else:
    print(f"\nCould not connect to any models. Please check the API key or model availability.")

Testing model: llama-3.1-8b-instant
Success! Connected with model: llama-3.1-8b-instant
The AI responded: Hello, connection established. I'm here to help

Perfect! We will use llama-3.1-8b-instant for our conversations.


## Core Conversation Management

Let us build our main conversation management system. This will track messages, maintain conversation history, and provide statistics about our chat sessions.

In [3]:
class ConversationManager:
    """
    Manages conversation history and provides various conversation utilities.
    """
    
    def __init__(self, max_turns=50):
        """
        Initialize the conversation manager.
        
        Args:
            max_turns (int): Maximum number of conversation turns to keep
        """
        self.messages = []
        self.max_turns = max_turns
        self.conversation_summary = ""
        self.metadata = {
            'created_at': None,
            'last_activity': None,
            'total_messages': 0,
            'user_messages': 0,
            'assistant_messages': 0
        }
    
    def add_message(self, role, content):
        """
        Add a message to the conversation.
        
        Args:
            role (str): Either 'user' or 'assistant'
            content (str): The message content
        """
        import datetime
        
        message = {
            'role': role,
            'content': content,
            'timestamp': datetime.datetime.now()
        }
        
        self.messages.append(message)
        
        # Update metadata
        if self.metadata['created_at'] is None:
            self.metadata['created_at'] = datetime.datetime.now()
        
        self.metadata['last_activity'] = datetime.datetime.now()
        self.metadata['total_messages'] += 1
        
        if role == 'user':
            self.metadata['user_messages'] += 1
        elif role == 'assistant':
            self.metadata['assistant_messages'] += 1
        
        # Keep conversation within limits
        self._manage_conversation_length()
    
    def _manage_conversation_length(self):
        """Keep conversation within max_turns limit."""
        if len(self.messages) > self.max_turns * 2:  # 2 messages per turn
            # Remove oldest messages but keep the conversation flow
            messages_to_remove = len(self.messages) - self.max_turns * 2
            self.messages = self.messages[messages_to_remove:]
    
    def get_conversation_for_api(self):
        """
        Get conversation in format suitable for API calls.
        
        Returns:
            list: List of message dictionaries without timestamps
        """
        return [{'role': msg['role'], 'content': msg['content']} for msg in self.messages]
    
    def display_conversation(self, last_n=None):
        """
        Display the conversation in a readable format.
        
        Args:
            last_n (int, optional): Number of recent messages to show
        """
        messages_to_show = self.messages[-last_n:] if last_n else self.messages
        
        if not messages_to_show:
            print("No messages in conversation yet.")
            return
        
        print("Conversation History:")
        print("-" * 50)
        
        for i, msg in enumerate(messages_to_show, 1):
            role_display = "User" if msg['role'] == 'user' else "Assistant"
            timestamp = msg['timestamp'].strftime("%H:%M:%S")
            print(f"{i}. [{timestamp}] {role_display}: {msg['content']}")
            print()
    
    def get_statistics(self):
        """
        Get conversation statistics.
        
        Returns:
            dict: Dictionary containing various statistics
        """
        if not self.messages:
            return {"message": "No conversation data available"}
        
        total_words = sum(len(msg['content'].split()) for msg in self.messages)
        total_chars = sum(len(msg['content']) for msg in self.messages)
        
        avg_message_length = total_words / len(self.messages) if self.messages else 0
        
        duration = None
        if self.metadata['created_at'] and self.metadata['last_activity']:
            duration = self.metadata['last_activity'] - self.metadata['created_at']
        
        return {
            'total_messages': self.metadata['total_messages'],
            'user_messages': self.metadata['user_messages'],
            'assistant_messages': self.metadata['assistant_messages'],
            'total_words': total_words,
            'total_characters': total_chars,
            'average_message_length': round(avg_message_length, 1),
            'conversation_duration': str(duration) if duration else "N/A",
            'created_at': self.metadata['created_at'].strftime("%Y-%m-%d %H:%M:%S") if self.metadata['created_at'] else None,
            'last_activity': self.metadata['last_activity'].strftime("%Y-%m-%d %H:%M:%S") if self.metadata['last_activity'] else None
        }
    
    def export_conversation(self, format_type='json'):
        """
        Export conversation in different formats.
        
        Args:
            format_type (str): Either 'json', 'text', or 'csv'
        
        Returns:
            str: Formatted conversation data
        """
        if format_type == 'json':
            import json
            export_data = {
                'metadata': self.metadata,
                'messages': [
                    {
                        'role': msg['role'],
                        'content': msg['content'],
                        'timestamp': msg['timestamp'].isoformat()
                    }
                    for msg in self.messages
                ]
            }
            return json.dumps(export_data, indent=2)
        
        elif format_type == 'text':
            lines = []
            lines.append("CONVERSATION EXPORT")
            lines.append("=" * 50)
            lines.append(f"Created: {self.metadata['created_at']}")
            lines.append(f"Messages: {self.metadata['total_messages']}")
            lines.append("")
            
            for msg in self.messages:
                timestamp = msg['timestamp'].strftime("%Y-%m-%d %H:%M:%S")
                role = msg['role'].upper()
                lines.append(f"[{timestamp}] {role}:")
                lines.append(f"{msg['content']}")
                lines.append("")
            
            return "\n".join(lines)
        
        elif format_type == 'csv':
            import csv
            import io
            
            output = io.StringIO()
            writer = csv.writer(output)
            writer.writerow(['timestamp', 'role', 'content', 'word_count'])
            
            for msg in self.messages:
                writer.writerow([
                    msg['timestamp'].isoformat(),
                    msg['role'],
                    msg['content'],
                    len(msg['content'].split())
                ])
            
            return output.getvalue()
        
        else:
            raise ValueError("format_type must be 'json', 'text', or 'csv'")

print("ConversationManager class is ready to use!")

ConversationManager class is ready to use!


### Testing the ConversationManager

Let us create a conversation manager and test its basic functionality.

In [4]:
# Create a conversation manager
conversation = ConversationManager(max_turns=20)

# Add some sample messages
conversation.add_message("user", "Hello! I need help with Python programming.")
conversation.add_message("assistant", "Hello! I would be happy to help you with Python programming. What specific topic would you like to learn about?")
conversation.add_message("user", "I want to understand how lists work in Python.")
conversation.add_message("assistant", "Lists in Python are ordered collections that can store multiple items. They are mutable, meaning you can change their contents after creation. Here are some basic operations:\n\n1. Creating a list: my_list = [1, 2, 3]\n2. Adding items: my_list.append(4)\n3. Accessing items: my_list[0] gets the first item\n4. Removing items: my_list.remove(2)")

# Display the conversation
conversation.display_conversation()

print("\n" + "="*50)
print("CONVERSATION STATISTICS:")
stats = conversation.get_statistics()
for key, value in stats.items():
    print(f"{key.replace('_', ' ').title()}: {value}")

Conversation History:
--------------------------------------------------
1. [16:38:12] User: Hello! I need help with Python programming.

2. [16:38:12] Assistant: Hello! I would be happy to help you with Python programming. What specific topic would you like to learn about?

3. [16:38:12] User: I want to understand how lists work in Python.

4. [16:38:12] Assistant: Lists in Python are ordered collections that can store multiple items. They are mutable, meaning you can change their contents after creation. Here are some basic operations:

1. Creating a list: my_list = [1, 2, 3]
2. Adding items: my_list.append(4)
3. Accessing items: my_list[0] gets the first item
4. Removing items: my_list.remove(2)


CONVERSATION STATISTICS:
Total Messages: 4
User Messages: 2
Assistant Messages: 2
Total Words: 88
Total Characters: 538
Average Message Length: 22.0
Conversation Duration: 0:00:00.000071
Created At: 2025-09-17 16:38:12
Last Activity: 2025-09-17 16:38:12


## AI-Powered Conversation Features

Now let us add AI capabilities to our conversation manager. This includes chat functionality and automatic summarization.

In [5]:
def chat_with_ai(conversation_manager, user_message, model=None):
    """
    Send a message to the AI and get a response, managing the conversation.
    
    Args:
        conversation_manager: ConversationManager instance
        user_message (str): The user's message
        model (str, optional): Model to use, defaults to DEFAULT_MODEL
    
    Returns:
        str: AI response
    """
    if model is None:
        model = DEFAULT_MODEL
    
    if not model:
        return "Error: No working model available. Please check your API connection."
    
    # Add user message to conversation
    conversation_manager.add_message("user", user_message)
    
    try:
        # Get current conversation for API
        messages = conversation_manager.get_conversation_for_api()
        
        # Call the AI
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            max_tokens=1000,
            temperature=0.7
        )
        
        ai_response = response.choices[0].message.content
        
        # Add AI response to conversation
        conversation_manager.add_message("assistant", ai_response)
        
        return ai_response
        
    except Exception as e:
        error_msg = f"Sorry, I encountered an error: {str(e)}"
        conversation_manager.add_message("assistant", error_msg)
        return error_msg

def create_conversation_summary(conversation_manager, max_length=200):
    """
    Create a summary of the conversation using AI.
    
    Args:
        conversation_manager: ConversationManager instance
        max_length (int): Maximum length of summary
    
    Returns:
        str: Conversation summary
    """
    if not conversation_manager.messages:
        return "No conversation to summarize."
    
    # Get the conversation text
    conversation_text = ""
    for msg in conversation_manager.messages:
        role = "User" if msg['role'] == 'user' else "Assistant"
        conversation_text += f"{role}: {msg['content']}\n"
    
    summary_prompt = f"""Please create a concise summary of this conversation in {max_length} words or less. Focus on the main topics discussed and key information exchanged:

{conversation_text}

Summary:"""
    
    try:
        response = client.chat.completions.create(
            model=DEFAULT_MODEL,
            messages=[{"role": "user", "content": summary_prompt}],
            max_tokens=max_length + 50,
            temperature=0.3
        )
        
        summary = response.choices[0].message.content.strip()
        conversation_manager.conversation_summary = summary
        return summary
        
    except Exception as e:
        return f"Could not create summary: {str(e)}"

print("AI conversation functions are ready!")

AI conversation functions are ready!


## Conversation Truncation Methods

When conversations get too long for AI models to handle, we need smart ways to shorten them while keeping the important context.

In [6]:
def truncate_by_turns(messages, max_turns=10):
    """
    Keep only the most recent conversation turns.
    
    Args:
        messages (list): List of message dictionaries
        max_turns (int): Maximum number of turns to keep
    
    Returns:
        list: Truncated messages
    """
    if len(messages) <= max_turns * 2:  # Each turn = user + assistant
        return messages
    
    # Keep the most recent messages
    return messages[-(max_turns * 2):]

def truncate_by_characters(messages, max_chars=4000):
    """
    Keep messages that fit within character limit, starting from most recent.
    
    Args:
        messages (list): List of message dictionaries
        max_chars (int): Maximum character count
    
    Returns:
        list: Truncated messages
    """
    if not messages:
        return []
    
    total_chars = 0
    truncated = []
    
    # Start from most recent and work backwards
    for message in reversed(messages):
        message_chars = len(message['content'])
        if total_chars + message_chars <= max_chars:
            truncated.insert(0, message)  # Insert at beginning to maintain order
            total_chars += message_chars
        else:
            break
    
    return truncated

def truncate_by_words(messages, max_words=800):
    """
    Keep messages that fit within word limit, starting from most recent.
    
    Args:
        messages (list): List of message dictionaries
        max_words (int): Maximum word count
    
    Returns:
        list: Truncated messages
    """
    if not messages:
        return []
    
    total_words = 0
    truncated = []
    
    # Start from most recent and work backwards
    for message in reversed(messages):
        message_words = len(message['content'].split())
        if total_words + message_words <= max_words:
            truncated.insert(0, message)  # Insert at beginning to maintain order
            total_words += message_words
        else:
            break
    
    return truncated

def smart_truncate_with_summary(conversation_manager, method='characters', limit=3000):
    """
    Truncate conversation and create summary of removed parts.
    
    Args:
        conversation_manager: ConversationManager instance
        method (str): 'characters', 'words', or 'turns'
        limit (int): Limit based on chosen method
    
    Returns:
        dict: Contains truncated messages and summary of removed content
    """
    original_messages = conversation_manager.messages
    
    if method == 'characters':
        truncated = truncate_by_characters(original_messages, limit)
    elif method == 'words':
        truncated = truncate_by_words(original_messages, limit)
    elif method == 'turns':
        truncated = truncate_by_turns(original_messages, limit)
    else:
        raise ValueError("Method must be 'characters', 'words', or 'turns'")
    
    # Check if any messages were removed
    if len(truncated) == len(original_messages):
        return {
            'truncated_messages': truncated,
            'summary': "No truncation was needed.",
            'removed_count': 0
        }
    
    # Create summary of removed messages
    removed_messages = original_messages[:len(original_messages) - len(truncated)]
    removed_count = len(removed_messages)
    
    # Create a simple summary of removed content
    if removed_messages:
        removed_content = []
        for msg in removed_messages:
            role = "User" if msg['role'] == 'user' else "Assistant"
            preview = msg['content'][:100] + "..." if len(msg['content']) > 100 else msg['content']
            removed_content.append(f"{role}: {preview}")
        
        summary = f"Removed {removed_count} earlier messages covering: " + "; ".join(removed_content[:3])
        if len(removed_content) > 3:
            summary += f" and {len(removed_content) - 3} more messages."
    else:
        summary = "No messages were removed."
    
    return {
        'truncated_messages': truncated,
        'summary': summary,
        'removed_count': removed_count
    }

print("Truncation methods are ready!")

Truncation methods are ready!


## Information Extraction from Conversations

Let us build a system to extract structured information from conversations using JSON schemas and AI-powered analysis.

In [7]:
# JSON Schemas for different types of information extraction

PERSONAL_INFO_SCHEMA = {
    "type": "object",
    "properties": {
        "name": {"type": "string", "description": "Person's full name"},
        "age": {"type": "integer", "minimum": 0, "maximum": 150},
        "email": {"type": "string", "format": "email"},
        "phone": {"type": "string"},
        "location": {"type": "string", "description": "City, state, or country"},
        "occupation": {"type": "string"},
        "interests": {
            "type": "array",
            "items": {"type": "string"},
            "description": "List of hobbies or interests"
        }
    },
    "required": []
}

CUSTOMER_SUPPORT_SCHEMA = {
    "type": "object",
    "properties": {
        "issue_type": {
            "type": "string",
            "enum": ["technical", "billing", "product", "account", "general"],
            "description": "Category of the customer issue"
        },
        "priority": {
            "type": "string",
            "enum": ["low", "medium", "high", "urgent"],
            "description": "Priority level of the issue"
        },
        "product_name": {"type": "string", "description": "Name of product discussed"},
        "customer_sentiment": {
            "type": "string",
            "enum": ["positive", "neutral", "negative", "frustrated"],
            "description": "Customer's emotional state"
        },
        "resolution_status": {
            "type": "string",
            "enum": ["resolved", "pending", "escalated", "unresolved"],
            "description": "Current status of the issue"
        },
        "key_points": {
            "type": "array",
            "items": {"type": "string"},
            "description": "Main points discussed in the conversation"
        }
    },
    "required": ["issue_type", "priority"]
}

LEARNING_SESSION_SCHEMA = {
    "type": "object",
    "properties": {
        "subject": {"type": "string", "description": "Main subject being learned"},
        "topics_covered": {
            "type": "array",
            "items": {"type": "string"},
            "description": "Specific topics discussed"
        },
        "difficulty_level": {
            "type": "string",
            "enum": ["beginner", "intermediate", "advanced"],
            "description": "Complexity level of the content"
        },
        "student_understanding": {
            "type": "string",
            "enum": ["poor", "fair", "good", "excellent"],
            "description": "How well the student seems to understand"
        },
        "questions_asked": {
            "type": "array",
            "items": {"type": "string"},
            "description": "Questions the student asked"
        },
        "concepts_explained": {
            "type": "array",
            "items": {"type": "string"},
            "description": "Key concepts that were explained"
        },
        "next_steps": {
            "type": "array",
            "items": {"type": "string"},
            "description": "Suggested next learning steps"
        }
    },
    "required": ["subject", "difficulty_level"]
}

# Schema validator class
class SchemaValidator:
    """Validates extracted information against JSON schemas."""
    
    def __init__(self):
        try:
            import jsonschema
            self.jsonschema = jsonschema
            self.validator_available = True
        except ImportError:
            print("Note: jsonschema library not available. Validation will be basic.")
            self.validator_available = False
    
    def validate(self, data, schema):
        """
        Validate data against a JSON schema.
        
        Args:
            data (dict): Data to validate
            schema (dict): JSON schema to validate against
        
        Returns:
            dict: Validation result with success status and any errors
        """
        if not self.validator_available:
            return {
                "valid": True,
                "errors": [],
                "message": "Validation skipped - jsonschema not available"
            }
        
        try:
            self.jsonschema.validate(instance=data, schema=schema)
            return {
                "valid": True,
                "errors": [],
                "message": "Validation successful"
            }
        except self.jsonschema.ValidationError as e:
            return {
                "valid": False,
                "errors": [str(e)],
                "message": f"Validation failed: {str(e)}"
            }
        except Exception as e:
            return {
                "valid": False,
                "errors": [str(e)],
                "message": f"Validation error: {str(e)}"
            }

# Create validator instance
validator = SchemaValidator()
print("JSON schemas and validator are ready!")

JSON schemas and validator are ready!


In [8]:
class InformationExtractor:
    """Extract structured information from conversations using AI and schemas."""
    
    def __init__(self, client, model=None):
        """
        Initialize the information extractor.
        
        Args:
            client: OpenAI client instance
            model (str): AI model to use for extraction
        """
        self.client = client
        self.model = model or DEFAULT_MODEL
        self.validator = SchemaValidator()
    
    def extract_information(self, conversation_manager, schema, schema_name="information"):
        """
        Extract structured information from a conversation.
        
        Args:
            conversation_manager: ConversationManager instance
            schema (dict): JSON schema defining the expected structure
            schema_name (str): Name describing what is being extracted
        
        Returns:
            dict: Extracted information with validation results
        """
        if not conversation_manager.messages:
            return {
                "success": False,
                "message": "No conversation data to extract from",
                "extracted_data": {},
                "validation": {"valid": False, "errors": ["No data"], "message": "No conversation"}
            }
        
        # Prepare conversation text
        conversation_text = self._format_conversation_for_extraction(conversation_manager)
        
        # Create extraction prompt
        prompt = self._create_extraction_prompt(conversation_text, schema, schema_name)
        
        try:
            # Call AI for extraction
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=1000,
                temperature=0.1  # Low temperature for more consistent extraction
            )
            
            # Parse the response
            extracted_text = response.choices[0].message.content.strip()
            extracted_data = self._parse_extraction_response(extracted_text)
            
            # Validate against schema
            validation_result = self.validator.validate(extracted_data, schema)
            
            return {
                "success": True,
                "message": "Information extracted successfully",
                "extracted_data": extracted_data,
                "validation": validation_result,
                "raw_response": extracted_text
            }
            
        except Exception as e:
            return {
                "success": False,
                "message": f"Extraction failed: {str(e)}",
                "extracted_data": {},
                "validation": {"valid": False, "errors": [str(e)], "message": "Extraction error"}
            }
    
    def _format_conversation_for_extraction(self, conversation_manager):
        """Format conversation for information extraction."""
        lines = []
        for msg in conversation_manager.messages:
            role = "User" if msg['role'] == 'user' else "Assistant"
            lines.append(f"{role}: {msg['content']}")
        return "\n".join(lines)
    
    def _create_extraction_prompt(self, conversation_text, schema, schema_name):
        """Create a prompt for information extraction."""
        # Convert schema to a readable description
        schema_description = self._schema_to_description(schema)
        
        prompt = f"""Please analyze the following conversation and extract {schema_name} as JSON.

CONVERSATION:
{conversation_text}

REQUIRED JSON STRUCTURE:
{schema_description}

INSTRUCTIONS:
- Only extract information that is explicitly mentioned in the conversation
- Use null for missing information
- Return ONLY valid JSON, no other text
- If no relevant information is found, return an empty object {{}}

JSON OUTPUT:"""
        
        return prompt
    
    def _schema_to_description(self, schema):
        """Convert JSON schema to readable description."""
        if schema.get("type") != "object":
            return str(schema)
        
        lines = ["{"]
        properties = schema.get("properties", {})
        required = schema.get("required", [])
        
        for prop, prop_schema in properties.items():
            prop_type = prop_schema.get("type", "any")
            description = prop_schema.get("description", "")
            is_required = prop in required
            
            line = f'  "{prop}": {prop_type}'
            if description:
                line += f' // {description}'
            if is_required:
                line += ' (REQUIRED)'
            
            lines.append(line)
        
        lines.append("}")
        return "\n".join(lines)
    
    def _parse_extraction_response(self, response_text):
        """Parse AI response to extract JSON data."""
        import json
        import re
        
        # Try to find JSON in the response
        json_match = re.search(r'\{.*\}', response_text, re.DOTALL)
        if json_match:
            json_text = json_match.group()
        else:
            json_text = response_text
        
        try:
            return json.loads(json_text)
        except json.JSONDecodeError:
            # If JSON parsing fails, try to clean and parse again
            cleaned = json_text.strip()
            if cleaned.startswith('```json'):
                cleaned = cleaned[7:]
            if cleaned.endswith('```'):
                cleaned = cleaned[:-3]
            
            try:
                return json.loads(cleaned)
            except json.JSONDecodeError:
                return {"error": "Could not parse JSON from AI response", "raw_response": response_text}

print("InformationExtractor class is ready!")

InformationExtractor class is ready!


## Complete System Demonstration

Let us put everything together and demonstrate the full conversation management and information extraction system.

In [9]:
# Create a new conversation manager for our demo
demo_conversation = ConversationManager(max_turns=15)

# Create information extractor
extractor = InformationExtractor(client, DEFAULT_MODEL)

print("DEMO 1: Customer Support Conversation")
print("=" * 50)

# Simulate a customer support conversation
support_messages = [
    ("user", "Hi, I'm having trouble with my SmartPhone Pro. The battery drains really fast and I'm quite frustrated about this."),
    ("assistant", "I understand your frustration with the SmartPhone Pro battery issue. This is definitely something we can help resolve. Can you tell me how long you've had the device and when you first noticed the battery draining quickly?"),
    ("user", "I bought it 3 months ago and the battery issue started about 2 weeks ago. It used to last all day but now it dies by noon."),
    ("assistant", "Thank you for that information. Battery drain that sudden usually indicates either a recent software update affecting performance or a specific app consuming more power. Have you installed any new apps recently or noticed any software updates?"),
    ("user", "Yes, there was a system update last month. Also, I installed a few gaming apps around the same time."),
    ("assistant", "That helps narrow it down. Gaming apps can be quite battery-intensive. Let me help you troubleshoot this. First, let's check your battery usage in Settings > Battery to see which apps are consuming the most power. Can you do that now?"),
    ("user", "I found the gaming app is using 45% of my battery! That's way too much."),
    ("assistant", "Perfect! That's definitely the culprit. For immediate relief, I recommend uninstalling or limiting the usage of that gaming app. For the system update, we can also optimize your settings. Would you like me to guide you through some battery optimization steps?"),
    ("user", "Yes, please help me optimize the settings. I want to keep the game but use it less."),
    ("assistant", "Excellent! Here's what we'll do: 1) Enable battery saver mode, 2) Reduce screen brightness and timeout, 3) Turn off background app refresh for non-essential apps, and 4) Limit the gaming app's background activity. This should significantly improve your battery life. Your issue should be resolved with these steps.")
]

# Add all messages to the conversation
for role, content in support_messages:
    demo_conversation.add_message(role, content)

# Display the conversation
demo_conversation.display_conversation()

print("\nEXTRACTING CUSTOMER SUPPORT INFORMATION:")
print("-" * 40)

# Extract customer support information
support_result = extractor.extract_information(
    demo_conversation, 
    CUSTOMER_SUPPORT_SCHEMA, 
    "customer support information"
)

if support_result["success"]:
    print("EXTRACTION SUCCESSFUL!")
    print(f"Validation: {'PASSED' if support_result['validation']['valid'] else 'FAILED'}")
    print("\nExtracted Information:")
    
    extracted = support_result["extracted_data"]
    for key, value in extracted.items():
        print(f"  {key.replace('_', ' ').title()}: {value}")
else:
    print(f"EXTRACTION FAILED: {support_result['message']}")

print("\nCONVERSATION STATISTICS:")
print("-" * 25)
stats = demo_conversation.get_statistics()
for key, value in stats.items():
    print(f"{key.replace('_', ' ').title()}: {value}")

DEMO 1: Customer Support Conversation
Conversation History:
--------------------------------------------------
1. [16:38:33] User: Hi, I'm having trouble with my SmartPhone Pro. The battery drains really fast and I'm quite frustrated about this.

2. [16:38:33] Assistant: I understand your frustration with the SmartPhone Pro battery issue. This is definitely something we can help resolve. Can you tell me how long you've had the device and when you first noticed the battery draining quickly?

3. [16:38:33] User: I bought it 3 months ago and the battery issue started about 2 weeks ago. It used to last all day but now it dies by noon.

4. [16:38:33] Assistant: Thank you for that information. Battery drain that sudden usually indicates either a recent software update affecting performance or a specific app consuming more power. Have you installed any new apps recently or noticed any software updates?

5. [16:38:33] User: Yes, there was a system update last month. Also, I installed a few gam

In [10]:
print("\n" + "="*60)
print("DEMO 2: Learning Session Conversation")
print("=" * 60)

# Create a new conversation for learning demo
learning_conversation = ConversationManager(max_turns=10)

# Simulate a Python learning session
learning_messages = [
    ("user", "I'm new to programming and want to learn Python. Can you help me understand what functions are?"),
    ("assistant", "Absolutely! I'd be happy to help you learn about Python functions. A function is like a recipe - it's a block of code that performs a specific task and can be used multiple times. Think of it as a mini-program within your program."),
    ("user", "That makes sense! Can you show me how to create a simple function?"),
    ("assistant", "Sure! Here's a simple function: \n\ndef greet(name):\n    return f'Hello, {name}!'\n\nThis function takes a name as input and returns a greeting. The 'def' keyword defines the function, 'greet' is the function name, and 'name' is a parameter."),
    ("user", "How do I use this function after creating it?"),
    ("assistant", "Great question! To use (or 'call') the function, you write: greet('Alice'). This would return 'Hello, Alice!'. You can call it with different names: greet('Bob'), greet('Charlie'), etc."),
    ("user", "What if I want a function that adds two numbers?"),
    ("assistant", "Perfect example! Here's an addition function:\n\ndef add_numbers(a, b):\n    return a + b\n\nYou would call it like: add_numbers(5, 3) which returns 8. Functions can take multiple parameters and perform calculations."),
    ("user", "This is really helpful! What should I learn next about functions?"),
    ("assistant", "You're doing great! Next, I'd suggest learning about: 1) Default parameters, 2) Functions that don't return values, 3) Local vs global variables, and 4) Lambda functions. We can explore any of these topics in more detail.")
]

# Add messages to learning conversation
for role, content in learning_messages:
    learning_conversation.add_message(role, content)

# Display the learning conversation
learning_conversation.display_conversation()

print("\nEXTRACTING LEARNING SESSION INFORMATION:")
print("-" * 45)

# Extract learning session information
learning_result = extractor.extract_information(
    learning_conversation, 
    LEARNING_SESSION_SCHEMA, 
    "learning session information"
)

if learning_result["success"]:
    print("EXTRACTION SUCCESSFUL!")
    print(f"Validation: {'PASSED' if learning_result['validation']['valid'] else 'FAILED'}")
    print("\nExtracted Learning Information:")
    
    extracted = learning_result["extracted_data"]
    for key, value in extracted.items():
        if isinstance(value, list):
            print(f"  {key.replace('_', ' ').title()}:")
            for item in value:
                print(f"    - {item}")
        else:
            print(f"  {key.replace('_', ' ').title()}: {value}")
else:
    print(f"EXTRACTION FAILED: {learning_result['message']}")

# Create a summary of the learning session
print("\nCREATING CONVERSATION SUMMARY:")
print("-" * 35)
summary = create_conversation_summary(learning_conversation, max_length=100)
print(f"Summary: {summary}")


DEMO 2: Learning Session Conversation
Conversation History:
--------------------------------------------------
1. [16:38:38] User: I'm new to programming and want to learn Python. Can you help me understand what functions are?

2. [16:38:38] Assistant: Absolutely! I'd be happy to help you learn about Python functions. A function is like a recipe - it's a block of code that performs a specific task and can be used multiple times. Think of it as a mini-program within your program.

3. [16:38:38] User: That makes sense! Can you show me how to create a simple function?

4. [16:38:38] Assistant: Sure! Here's a simple function: 

def greet(name):
    return f'Hello, {name}!'

This function takes a name as input and returns a greeting. The 'def' keyword defines the function, 'greet' is the function name, and 'name' is a parameter.

5. [16:38:38] User: How do I use this function after creating it?

6. [16:38:38] Assistant: Great question! To use (or 'call') the function, you write: greet('Ali

In [11]:
print("\n" + "="*60)
print("DEMO 3: Conversation Truncation Methods")
print("=" * 60)

# Create a longer conversation to demonstrate truncation
long_conversation = ConversationManager(max_turns=20)

# Add many messages to simulate a long conversation
long_messages = [
    ("user", "Hello, I need help with my computer."),
    ("assistant", "Hello! I'd be happy to help you with your computer issue. What specific problem are you experiencing?"),
    ("user", "My computer is running very slowly and sometimes freezes."),
    ("assistant", "I understand that's frustrating. Slow performance and freezing can have several causes. Let's troubleshoot this step by step."),
    ("user", "Okay, what should I check first?"),
    ("assistant", "First, let's check how much storage space you have available. Low disk space is a common cause of slow performance."),
    ("user", "I checked and I have about 50GB free out of 500GB."),
    ("assistant", "That's actually quite good - 50GB should be plenty of free space. Let's check your Task Manager to see if any programs are using too much memory or CPU."),
    ("user", "I see Chrome is using a lot of memory - about 2GB."),
    ("assistant", "Chrome can indeed use a lot of memory, especially with many tabs open. Try closing some tabs or restarting Chrome to see if that helps."),
    ("user", "I closed Chrome and the computer seems a bit faster now."),
    ("assistant", "Great! That's a good sign. Now let's check for any background programs that might be consuming resources unnecessarily."),
    ("user", "How do I check for background programs?"),
    ("assistant", "In Task Manager, look at the Processes tab. Sort by CPU or Memory usage to see which programs are using the most resources."),
    ("user", "I see something called 'Windows Update' using CPU. Is that normal?"),
    ("assistant", "Yes, Windows Update using CPU is completely normal. It means your system is downloading or installing updates, which is important for security and performance.")
]

# Add all messages
for role, content in long_messages:
    long_conversation.add_message(role, content)

print(f"Original conversation has {len(long_conversation.messages)} messages")
print(f"Total words: {sum(len(msg['content'].split()) for msg in long_conversation.messages)}")
print(f"Total characters: {sum(len(msg['content']) for msg in long_conversation.messages)}")

print("\nTESTING TRUNCATION METHODS:")
print("-" * 35)

# Test truncation by turns
print("1. Truncation by turns (keep last 4 turns = 8 messages):")
truncated_turns = truncate_by_turns(long_conversation.messages, max_turns=4)
print(f"   Result: {len(truncated_turns)} messages kept")

# Test truncation by characters
print("\n2. Truncation by characters (keep messages under 1000 chars):")
truncated_chars = truncate_by_characters(long_conversation.messages, max_chars=1000)
print(f"   Result: {len(truncated_chars)} messages kept")
char_count = sum(len(msg['content']) for msg in truncated_chars)
print(f"   Total characters: {char_count}")

# Test truncation by words
print("\n3. Truncation by words (keep messages under 200 words):")
truncated_words = truncate_by_words(long_conversation.messages, max_words=200)
print(f"   Result: {len(truncated_words)} messages kept")
word_count = sum(len(msg['content'].split()) for msg in truncated_words)
print(f"   Total words: {word_count}")

# Test smart truncation with summary
print("\n4. Smart truncation with summary:")
smart_result = smart_truncate_with_summary(long_conversation, method='characters', limit=800)
print(f"   Messages kept: {len(smart_result['truncated_messages'])}")
print(f"   Messages removed: {smart_result['removed_count']}")
print(f"   Summary of removed content: {smart_result['summary'][:100]}...")

print("\nSample of truncated conversation (last 3 messages):")
for i, msg in enumerate(smart_result['truncated_messages'][-3:], 1):
    role = "User" if msg['role'] == 'user' else "Assistant"
    content = msg['content'][:100] + "..." if len(msg['content']) > 100 else msg['content']
    print(f"   {i}. {role}: {content}")


DEMO 3: Conversation Truncation Methods
Original conversation has 16 messages
Total words: 248
Total characters: 1415

TESTING TRUNCATION METHODS:
-----------------------------------
1. Truncation by turns (keep last 4 turns = 8 messages):
   Result: 8 messages kept

2. Truncation by characters (keep messages under 1000 chars):
   Result: 10 messages kept
   Total characters: 949

3. Truncation by words (keep messages under 200 words):
   Result: 12 messages kept
   Total words: 197

4. Smart truncation with summary:
   Messages kept: 8
   Messages removed: 8
   Summary of removed content: Removed 8 earlier messages covering: User: Hello, I need help with my computer.; Assistant: Hello! I...

Sample of truncated conversation (last 3 messages):
   1. Assistant: In Task Manager, look at the Processes tab. Sort by CPU or Memory usage to see which programs are us...
   2. User: I see something called 'Windows Update' using CPU. Is that normal?
   3. Assistant: Yes, Windows Update using CP

## Interactive Chat Demo

Let us create an interactive chat session with the AI and demonstrate real-time conversation management.

In [12]:
# Create a new conversation for interactive demo
interactive_conversation = ConversationManager(max_turns=10)

print("INTERACTIVE CHAT DEMO")
print("=" * 40)
print("Note: This demo shows how the system would work interactively.")
print("In a real application, you would get user input dynamically.\n")

# Simulate an interactive conversation about programming
demo_questions = [
    "Can you explain what object-oriented programming is?",
    "What are the main principles of OOP?",
    "Can you give me a simple example of a class in Python?",
    "How is inheritance useful in programming?"
]

print("Simulating interactive chat session...")
print("-" * 40)

for i, question in enumerate(demo_questions, 1):
    print(f"\n[Turn {i}]")
    print(f"User: {question}")
    
    # Get AI response using our chat function
    ai_response = chat_with_ai(interactive_conversation, question)
    print(f"Assistant: {ai_response[:200]}{'...' if len(ai_response) > 200 else ''}")
    
    # Show conversation stats after each turn
    stats = interactive_conversation.get_statistics()
    print(f"[Stats: {stats['total_messages']} messages, {stats['total_words']} words]")

print("\n" + "="*50)
print("FINAL CONVERSATION ANALYSIS")
print("="*50)

# Display full conversation
print("\nFULL CONVERSATION:")
interactive_conversation.display_conversation()

# Extract learning information
print("\nEXTRACTING INFORMATION:")
learning_info = extractor.extract_information(
    interactive_conversation, 
    LEARNING_SESSION_SCHEMA, 
    "programming learning session"
)

if learning_info["success"]:
    print("Successfully extracted learning session information:")
    extracted = learning_info["extracted_data"]
    for key, value in extracted.items():
        if isinstance(value, list) and value:
            print(f"  {key.replace('_', ' ').title()}:")
            for item in value[:3]:  # Show first 3 items
                print(f"    - {item}")
            if len(value) > 3:
                print(f"    ... and {len(value) - 3} more")
        else:
            print(f"  {key.replace('_', ' ').title()}: {value}")

# Create summary
print(f"\nCONVERSATION SUMMARY:")
summary = create_conversation_summary(interactive_conversation, max_length=150)
print(f"{summary}")

# Final statistics
print(f"\nFINAL STATISTICS:")
final_stats = interactive_conversation.get_statistics()
for key, value in final_stats.items():
    print(f"  {key.replace('_', ' ').title()}: {value}")

print(f"\nDemonstration complete! The conversation management system successfully:")
print(f"- Tracked {final_stats['total_messages']} messages across {final_stats['total_messages']//2} conversation turns")
print(f"- Maintained conversation history and metadata")
print(f"- Integrated with AI for natural conversation flow")
print(f"- Extracted structured information using JSON schemas")
print(f"- Generated conversation summaries")
print(f"- Provided comprehensive analytics and statistics")

INTERACTIVE CHAT DEMO
Note: This demo shows how the system would work interactively.
In a real application, you would get user input dynamically.

Simulating interactive chat session...
----------------------------------------

[Turn 1]
User: Can you explain what object-oriented programming is?
Assistant: **Object-Oriented Programming (OOP)**

Object-Oriented Programming (OOP) is a programming paradigm that revolves around the concept of objects and classes. It's a...
[Stats: 2 messages, 417 words]

[Turn 2]
User: What are the main principles of OOP?
Assistant: **Main Principles of Object-Oriented Programming (OOP)**
--------------------------------------------------------

Object-Oriented Programming (OOP) is based on several fundamental principles that hel...
[Stats: 4 messages, 938 words]

[Turn 3]
User: Can you give me a simple example of a class in Python?
Assistant: **Simple Class Example in Python**

Here's a simple example of a `Person` class in Python:
```python
class Person:


## Summary and Conclusion

This notebook demonstrates a comprehensive conversation management system with the following key features:

### Core Components

1. **ConversationManager Class**
   - Tracks conversation history and metadata
   - Manages conversation length and statistics
   - Supports multiple export formats (JSON, text, CSV)
   - Provides comprehensive analytics

2. **AI Integration**
   - Seamless integration with Groq AI models
   - Automatic conversation flow management
   - AI-powered summarization capabilities
   - Error handling and fallback mechanisms

3. **Truncation Methods**
   - Truncation by conversation turns
   - Truncation by character count
   - Truncation by word count
   - Smart truncation with summaries of removed content

4. **Information Extraction**
   - JSON schema-based extraction
   - Support for multiple conversation types (customer support, learning, personal info)
   - Validation of extracted information
   - Flexible schema system for different use cases

### Key Benefits

- **Scalable**: Handles conversations of any length with intelligent truncation
- **Structured**: Extracts meaningful information in standardized formats
- **Analytical**: Provides detailed statistics and insights
- **Flexible**: Adapts to different conversation types and schemas
- **Robust**: Includes error handling and validation

### Use Cases

This system can be applied to:
- Customer support analytics
- Educational conversation tracking
- Personal information extraction
- Chat bot conversation management
- Research and conversation analysis

The combination of conversation management, AI integration, and information extraction creates a powerful framework for building sophisticated conversational applications.